<a href="https://colab.research.google.com/github/ktanguy/University_chatbot_assistant/blob/main/university_chatbot_RAG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%bash
# Clean and align base libs (do this once per runtime)
pip uninstall -y numpy pandas scikit-learn || true
python -m pip install -U pip
pip install "numpy==1.26.4" "pandas==2.2.2" "scikit-learn==1.5.2"


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached scikit_learn-1.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.9 MB)



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 11.0.3 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [1]:
import numpy as np, pandas as pd, sklearn
print("NumPy       :", np.__version__)   # 1.26.4
print("Pandas      :", pd.__version__)   # 2.2.2
print("scikit-learn:", sklearn.__version__)  # 1.5.2


NumPy       : 1.26.4
Pandas      : 2.2.2
scikit-learn: 1.5.2


In [13]:
# --- Install & start Ollama in Colab (safe version) ---
import time, subprocess, requests, os

# 1) Install Ollama (idempotent)
!curl -fsSL https://ollama.com/install.sh | sh

# 2) Start the daemon in the background
_ = subprocess.Popen(["bash","-lc","nohup ollama serve > /tmp/ollama.log 2>&1 &"])
time.sleep(3)

# 3) Pull models
!ollama pull llama3.2:1b-instruct
!ollama pull qwen2.5:1.5b-instruct

# 4) Wait for HTTP endpoint
def wait_for_ollama(timeout=90):
    url = "http://127.0.0.1:11434/api/tags"
    t0 = time.time()
    while True:
        try:
            r = requests.get(url, timeout=5)
            if r.status_code == 200:
                print("✅ Ollama is running.")
                return
        except Exception:
            pass
        if time.time() - t0 > timeout:
            try:
                print("\n--- /tmp/ollama.log (tail) ---")
                print("".join(open("/tmp/ollama.log","r").readlines()[-200:]))
            except: pass
            raise RuntimeError("Ollama server did not start in time.")
        time.sleep(2)

wait_for_ollama()
print("✅ Models pulled: llama3.2:1b-instruct, qwen2.5:1.5b-instruct")


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

Error: pull model manifest: file does not exist

✅ Ollama is running.
✅ Models pulled: llama3.2:1b-instruct, qwen2.5:1.5b-instruct


In [15]:
from google.colab import drive
drive.mount('/content/drive')

import os, pandas as pd

# 💾 Save everything in Google Drive
BASE_DIR  = "/content/drive/MyDrive/alu_chatbot_rag"
DATA_DIR  = os.path.join(BASE_DIR, "data")
OUT_DIR   = os.path.join(BASE_DIR, "outputs")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# Update dataset paths
INTENTS_JSON = "/content/drive/MyDrive/UniversityChatbot/intents_merged_tagged.json"
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
TEST_CSV  = os.path.join(DATA_DIR, "test.csv")

print("✅ Drive mounted and paths set:")
print("BASE_DIR :", BASE_DIR)
print("DATA_DIR :", DATA_DIR)
print("OUT_DIR  :", OUT_DIR)
print("INTENTS_JSON:", INTENTS_JSON)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted and paths set:
BASE_DIR : /content/drive/MyDrive/alu_chatbot_rag
DATA_DIR : /content/drive/MyDrive/alu_chatbot_rag/data
OUT_DIR  : /content/drive/MyDrive/alu_chatbot_rag/outputs
INTENTS_JSON: /content/drive/MyDrive/UniversityChatbot/intents_merged_tagged.json


In [16]:
import json, random, csv, re, html

assert os.path.exists(INTENTS_JSON), f"Missing: {INTENTS_JSON}"

with open(INTENTS_JSON, "r", encoding="utf-8") as f:
    intents = json.load(f)

def normalize_text(s: str) -> str:
    s = html.unescape(str(s or ""))
    s = re.sub(r"<[^>]+>", " ", s)                # strip HTML
    s = re.sub(r"\s{2,}", " ", s).strip()         # collapse spaces
    return s

pairs = []
for it in intents.get("intents", []):
    texts = it.get("text") or []
    responses = it.get("responses") or []
    if not texts or not responses:
        continue
    resp = normalize_text(responses[0])
    for t in texts:
        t = normalize_text(t)
        if t and resp:
            pairs.append((t, resp))

print("Built pairs:", len(pairs))

# train/test split (80/20) only if CSVs not present
if not (os.path.exists(TRAIN_CSV) and os.path.exists(TEST_CSV)):
    random.shuffle(pairs)
    split = int(0.8 * len(pairs))
    train_pairs, test_pairs = pairs[:split], pairs[split:]
    with open(TRAIN_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["user","bot"]); w.writerows(train_pairs)
    with open(TEST_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["user","bot"]); w.writerows(test_pairs)

print("Train CSV:", TRAIN_CSV, "→", len(pd.read_csv(TRAIN_CSV)))
print("Test  CSV:", TEST_CSV, "→", len(pd.read_csv(TEST_CSV)))


Built pairs: 412
Train CSV: /content/drive/MyDrive/alu_chatbot_rag/data/train.csv → 326
Test  CSV: /content/drive/MyDrive/alu_chatbot_rag/data/test.csv → 83


In [17]:
import pandas as pd, re
from collections import Counter

PLACEHOLDERS = [
    r"ADD\s+YOUR.*", r"ADD\s+YOU\s+OWN\s+ANSWERS.*", r"ADD\s+YOU\s+GOOGLE\s+MAP.*",
    r"CLICK\s+HERE.*", r"\bhere\b\s*$"
]
TAG_RE = re.compile(r"<[^>]+>")
JUNK = ["target=\"_blank\"", "href=", "</a>", "<a", "http://", "https://"]

def looks_placeholder(s: str) -> bool:
    if not s or not s.strip(): return True
    if TAG_RE.search(s): return True
    if any(j in s for j in JUNK): return True
    for p in PLACEHOLDERS:
        if re.search(p, s, re.I): return True
    return False

ALU_KB = {
    "admission":  "Apply via the official ALU portal. Submit all required documents before the deadline.",
    "fees":       "Tuition varies by programme and year. See Student Portal > Finance for current fees.",
    "scholarship":"Scholarships/financial aid may be available—check the finance/admissions pages.",
    "hostel":     "Limited on-campus or partner housing may be available. Contact Student Services.",
    "calendar":   "See the Academic Calendar on the Student Portal for term dates and exams.",
    "timetable":  "Your class timetable is available on the Student Portal.",
    "programme":  "See the Programmes page for offerings and entry requirements.",
    "library":    "Use student credentials to access the library portal and digital resources.",
    "canteen":    "On-campus food options are available; details vary by term.",
    "contact":    "Use the Student Portal or official Contact page for support channels.",
    "general":    "Please refer to the Student Portal or Student Services for the latest information."
}

ROUTES = [
    ("admission",  ["admission","admissions","apply","application","offer","enrol","enroll"]),
    ("fees",       ["fee","fees","tuition","invoice","payment","pay","bank","cost"]),
    ("scholarship",["scholarship","financial aid","bursary","aid"]),
    ("hostel",     ["hostel","housing","accommodation","residence","dorm"]),
    ("calendar",   ["calendar","academic calendar"]),
    ("timetable",  ["timetable","schedule","class time","class schedule","time table"]),
    ("programme",  ["programme","program","course","degree","major","specialization"]),
    ("library",    ["library","ebook","database","journal","catalog"]),
    ("canteen",    ["canteen","cafeteria","food","menu","dining"]),
    ("contact",    ["contact","email","phone","helpdesk","support","whatsapp"])
]

def route(user: str) -> str:
    u = (user or "").lower()
    for key, kws in ROUTES:
        if any(kw in u for kw in kws):
            return ALU_KB[key]
    return ALU_KB["general"]

def clean_frame(path: str):
    df = pd.read_csv(path).dropna().astype(str)
    before = len(df)
    df = df.drop_duplicates(subset=["user","bot"]).reset_index(drop=True)
    bots = []
    for u, b in zip(df["user"], df["bot"]):
        s = TAG_RE.sub("", b)
        for p in PLACEHOLDERS: s = re.sub(p, "", s, flags=re.I)
        for j in JUNK: s = s.replace(j, "")
        s = re.sub(r"\s{2,}", " ", s).strip()
        if looks_placeholder(s): s = route(u)
        bots.append(s)
    df["bot"] = bots
    df.to_csv(path, index=False)
    print(f"Cleaned {path}: {before} -> {len(df)} rows")
    return df

train_df = clean_frame(TRAIN_CSV)
test_df  = clean_frame(TEST_CSV)

display(train_df.sample(5, random_state=0))


Cleaned /content/drive/MyDrive/alu_chatbot_rag/data/train.csv: 326 -> 326 rows
Cleaned /content/drive/MyDrive/alu_chatbot_rag/data/test.csv: 83 -> 83 rows


,user,bot
122,scholarship civil,Many government scholarships are supported by ...
66,information about fee,For Fee detail visit
142,fees for Ac room for girls,For Fee detail visit
245,hostel college distance,For hostel detail visit
146,max number of students,"For IT, Computer and extc 60 per branch and se..."


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

KB_QS, KB_AS = train_df["user"].tolist(), train_df["bot"].tolist()

# Character + word n-grams help with noisy text
vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1,2),
    min_df=1,
    strip_accents="unicode"
)
X = vectorizer.fit_transform(KB_QS)
nn = NearestNeighbors(n_neighbors=3, metric="cosine").fit(X)

def retrieve_context(query, topk=3):
    qv = vectorizer.transform([query])
    dist, idx = nn.kneighbors(qv, n_neighbors=topk, return_distance=True)
    contexts = []
    for d,i in zip(dist[0], idx[0]):
        contexts.append(dict(q=KB_QS[i], a=KB_AS[i], cos_sim=1.0-float(d)))
    return contexts

# quick sanity
retrieve_context("How do I apply to ALU?", topk=2)


[{'q': 'how to call you',
  'a': 'You can contact at: NUMBER',
  'cos_sim': 0.3507761562366105},
 {'q': 'how to contact college',
  'a': 'You can contact at: NUMBER',
  'cos_sim': 0.3484309528328928}]

In [27]:
# ---- Ollama client (dual-endpoint + model sanity) ----
import requests, time, json, os

OLLAMA_URL = "http://127.0.0.1:11434"

def _ollama_tags():
    try:
        r = requests.get(f"{OLLAMA_URL}/api/tags", timeout=10)
        r.raise_for_status()
        return [m["name"] for m in r.json().get("models", [])]
    except Exception as e:
        print("⚠️ Could not read /api/tags:", e)
        return []

def ensure_model_available(model_name: str):
    tags = _ollama_tags()
    if model_name not in tags:
        raise RuntimeError(
            f"Model '{model_name}' not found in Ollama. Available:\n- " + "\n- ".join(tags)
        )

def ollama_chat(model: str, system_prompt: str, user_prompt: str,
                temperature=0.2, retries=3):
    """
    Robust call:
      1) Prefers /api/generate (single-prompt format).
      2) On 404, falls back to /api/chat (messages format).
      3) Retries transient errors.
    """
    ensure_model_available(model)

    gen_payload = {
        "model": model,
        "prompt": f"<|system|>\n{system_prompt}\n<|user|>\n{user_prompt}\n<|assistant|>\n",
        "options": {"temperature": temperature},
        "stream": False
    }
    chat_payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt}
        ],
        "options": {"temperature": temperature},
        "stream": False
    }

    last_err = None
    for attempt in range(1, retries+1):
        try:
            # Try /api/generate first
            r = requests.post(f"{OLLAMA_URL}/api/generate", json=gen_payload, timeout=120)
            if r.status_code == 404:
                # Fallback to /api/chat
                r = requests.post(f"{OLLAMA_URL}/api/chat", json=chat_payload, timeout=120)
            r.raise_for_status()
            data = r.json()
            # unify outputs
            if "response" in data:
                return (data.get("response") or "").strip()
            if "message" in data and isinstance(data["message"], dict):
                return (data["message"].get("content") or "").strip()
            return ""
        except Exception as e:
            print(f"⚠️ Ollama call failed (attempt {attempt}/{retries}): {e}")
            last_err = e
            time.sleep(2)
    raise last_err


In [29]:
# Preflight sanity using the model that actually exists
print("Ollama models:", _ollama_tags())  # should list: ['qwen2.5:1.5b-instruct']

SYSTEM_PROMPT = "You are a helpful ALU assistant. Be brief and accurate."
test_ans = ollama_chat("qwen2.5:1.5b-instruct", SYSTEM_PROMPT, "Say 'ready' if you can hear me.", temperature=0.0)
print("Test answer:", test_ans)


Ollama models: ['qwen2.5:1.5b-instruct']
Test answer: Ready.


In [28]:
# ---- Preflight sanity checks ----
print("Ollama models:", _ollama_tags())  # do you see: 'llama3.2:1b-instruct', 'qwen2.5:1.5b-instruct' ?

# If you see a different tag (e.g., quantized variant), use that exact string in your configs.
# Quick echo test (no retrieval):
SYSTEM_PROMPT = "You are a helpful ALU assistant. Be brief and accurate."
try:
    test_ans = ollama_chat("llama3.2:1b-instruct", SYSTEM_PROMPT, "Say 'ready' if you can hear me.", temperature=0.0)
    print("Test answer:", test_ans)
except Exception as e:
    print("Preflight test failed:", e)


Ollama models: ['qwen2.5:1.5b-instruct']
Preflight test failed: Model 'llama3.2:1b-instruct' not found in Ollama. Available:
- qwen2.5:1.5b-instruct


In [24]:
import sacrebleu, numpy as np

def bleu_like(preds, refs):
    # sacrebleu expects list of sys outputs and list of reference lists
    return float(sacrebleu.corpus_bleu(preds, [refs]).score)

def f1_token(preds, refs):
    vals=[]
    for p,r in zip(preds, refs):
        ps, rs = set(p.lower().split()), set(r.lower().split())
        if not ps or not rs: vals.append(0.0); continue
        inter = len(ps & rs); prec = inter/len(ps); rec = inter/len(rs)
        vals.append(0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec))
    return float(np.mean(vals))

def evaluate_model(model_id: str, use_retrieval=True, temperature=0.2, max_examples=80):
    df = test_df.sample(min(max_examples, len(test_df)), random_state=0).reset_index(drop=True)
    preds, refs, lat = [], [], []
    for _, row in df.iterrows():
        q, ref = row["user"], row["bot"]
        t0 = time.time()
        if use_retrieval:
            ctx = retrieve_context(q, topk=3)
            uprompt = build_prompt(q, ctx)
        else:
            uprompt = q
        pred = ollama_chat(model_id, SYSTEM_PROMPT, uprompt, temperature=temperature)
        lat.append(time.time()-t0)
        preds.append(pred); refs.append(ref)
    return dict(
        BLEU=round(bleu_like(preds, refs), 2),
        F1=round(f1_token(preds, refs), 3),
        latency_s=round(float(np.mean(lat)), 3),
        n=len(preds),
        preds=preds, refs=refs, sample=df["user"].tolist()
    )


In [30]:
import pandas as pd, os

# ✅ 1. Detect which models are actually installed in Ollama
AVAILABLE = _ollama_tags()  # e.g. ['qwen2.5:1.5b-instruct']
print("Available Ollama models:", AVAILABLE)

# ✅ 2. Build only valid configs
CANDIDATES = [m for m in AVAILABLE if any(x in m for x in [
    "qwen2.5:1.5b-instruct", "llama3.2:1b-instruct"
])]
if not CANDIDATES:
    raise RuntimeError(f"No valid models found in Ollama. Installed: {AVAILABLE}")

configs = []
for m in CANDIDATES:
    for t in [0.2, 0.5]:
        for ret in [True, False]:
            configs.append(dict(model=m, temp=t, retrieval=ret))

# ✅ 3. Run experiments
records, all_runs = [], {}
print(f"Running {len(configs)} experiments …")

for cfg in configs:
    print(f"\n▶ {cfg}")
    res = evaluate_model(cfg["model"], use_retrieval=cfg["retrieval"], temperature=cfg["temp"], max_examples=80)
    row = dict(model=cfg["model"], temp=cfg["temp"], retrieval=cfg["retrieval"],
               BLEU=res["BLEU"], F1=res["F1"], latency_s=res["latency_s"], n=res["n"])
    records.append(row)
    all_runs[(cfg["model"], cfg["temp"], cfg["retrieval"])] = res

# ✅ 4. Save & display results
res_df = pd.DataFrame(records).sort_values(by=["BLEU","F1"], ascending=False).reset_index(drop=True)
display(res_df)

os.makedirs(OUT_DIR, exist_ok=True)
res_path = os.path.join(OUT_DIR, "rag_experiments.csv")
res_df.to_csv(res_path, index=False)
print("Saved results to:", res_path)

# ✅ 5. Pick the best run for deployment
best = res_df.iloc[0].to_dict()
print("\n🏆 Best configuration:", best)
best_key = (best["model"], best["temp"], best["retrieval"])


Available Ollama models: ['qwen2.5:1.5b-instruct']
Running 4 experiments …

▶ {'model': 'qwen2.5:1.5b-instruct', 'temp': 0.2, 'retrieval': True}

▶ {'model': 'qwen2.5:1.5b-instruct', 'temp': 0.2, 'retrieval': False}

▶ {'model': 'qwen2.5:1.5b-instruct', 'temp': 0.5, 'retrieval': True}

▶ {'model': 'qwen2.5:1.5b-instruct', 'temp': 0.5, 'retrieval': False}


,model,temp,retrieval,BLEU,F1,latency_s,n
0,qwen2.5:1.5b-instruct,0.2,True,5.17,0.193,14.520,80
1,qwen2.5:1.5b-instruct,0.5,True,3.04,0.176,15.198,80
2,qwen2.5:1.5b-instruct,0.2,False,0.06,0.067,16.887,80
3,qwen2.5:1.5b-instruct,0.5,False,0.05,0.068,18.421,80


Saved results to: /content/drive/MyDrive/alu_chatbot_rag/outputs/rag_experiments.csv

🏆 Best configuration: {'model': 'qwen2.5:1.5b-instruct', 'temp': 0.2, 'retrieval': True, 'BLEU': 5.17, 'F1': 0.193, 'latency_s': 14.52, 'n': 80}


In [31]:
best_res = all_runs[best_key]
peek = pd.DataFrame({
    "user": best_res["sample"][:10],
    "ref":  best_res["refs"][:10],
    "pred": best_res["preds"][:10],
})
peek


,user,ref,pred
0,scholarship for computer engineering,Many government scholarships are supported by ...,There is no specific information provided abou...
1,contact info,You can contact at: NUMBER,You can contact the organization using the fol...
2,seat allotment,"For IT, Computer and extc 60 per branch and se...",The context provided does not directly answer ...
3,Mess TimeTable,Timetable Link visit,The Mess TimeTable can be found by visiting th...
4,when should i attend college,College is open 8am-5pm Monday-Saturday!,You should come to college on weekdays from Mo...
5,please provide sports and games information,Our university encourages all-round developmen...,Our university promotes comprehensive student ...
6,uniform,ENTER YOUR OWN UNIVERSITY UNIFORM CIRCULER,The context provided is a circular poster or f...
7,your name,You can call me Mind Reader.,"My name is Mind Reader, and I am a helpful ass..."
8,what is the college hours,College is open 8am-5pm Monday-Saturday!,The college hours are not specified in the giv...
9,syllabus,Timetable provide direct to the students OR To...,The syllabus can be accessed directly through ...


In [32]:
import gradio as gr

BEST_MODEL = best["model"]
BEST_TEMP  = float(best["temp"])
BEST_RET   = bool(best["retrieval"])

def respond(user_input):
    q = (user_input or "").strip()
    if not q:
        return "Ask about admissions, fees, scholarships, hostels, programmes, calendar, library, or contacts."
    if BEST_RET:
        ctx = retrieve_context(q, topk=3)
        uprompt = build_prompt(q, ctx)
    else:
        uprompt = q
    ans = ollama_chat(BEST_MODEL, SYSTEM_PROMPT, uprompt, temperature=BEST_TEMP)
    return ans

demo = gr.Interface(
    fn=respond,
    inputs=gr.Textbox(placeholder="e.g., How do I apply to ALU?"),
    outputs="text",
    title=f"ALU Chatbot — RAG (Model={BEST_MODEL}, temp={BEST_TEMP}, retrieval={BEST_RET})",
    description="Answers ALU questions grounded on retrieved Q/A from the dataset.",
)
demo.launch(share=True)


IMPORTANT: You are using gradio version 4.31.5, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c84790431c0f44636.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
